In [1]:
from scipy.integrate import odeint
import numpy as np

def lorenz(w, t, p, r, b):
    x, y, z = w.tolist()
    return p*(y-x), x*(r-z)-y, x*y-b*z

t = np.arange(0, 30, 0.02)
track1 = odeint(lorenz, (0.0, 1.00, 0.0), t, args=(10.0, 28.0, 3.0))
track2 = odeint(lorenz, (0.0, 1.01, 0.0), t, args=(10.0, 28.0, 3.0))

In [4]:
def mass_spring_damper(xu, t, m, k, b, F):
    x, u = xu.tolist()
    dx = u
    du = (F - k*x - b*u) / m
    return dx, du
    
m, b, k, F = 1.0, 10.0, 20.0, 1.0
init_status = 0.0, 0.0
args = m, k, b, F
t = np.arange(0, 2, 0.01)
result = odeint(mass_spring_damper, init_status, t, args)
print result

[[  0.00000000e+00   0.00000000e+00]
 [  4.83676790e-05   9.51307513e-03]
 [  1.87185966e-04   1.81027688e-02]
 [  4.07583661e-04   2.58406027e-02]
 [  7.01379356e-04   3.27929077e-02]
 [  1.06102752e-03   3.90212196e-02]
 [  1.47957816e-03   4.45825754e-02]
 [  1.95063426e-03   4.95298266e-02]
 [  2.46829516e-03   5.39120551e-02]
 [  3.02714337e-03   5.77746717e-02]
 [  3.62219720e-03   6.11597646e-02]
 [  4.24887777e-03   6.41063426e-02]
 [  4.90298319e-03   6.66505264e-02]
 [  5.58065894e-03   6.88257675e-02]
 [  6.27837239e-03   7.06630362e-02]
 [  6.99288907e-03   7.21909981e-02]
 [  7.72125070e-03   7.34361769e-02]
 [  8.46075273e-03   7.44231209e-02]
 [  9.20892747e-03   7.51745286e-02]
 [  9.96352715e-03   7.57113748e-02]
 [  1.07225039e-02   7.60530601e-02]
 [  1.14839973e-02   7.62175033e-02]
 [  1.22463185e-02   7.62212607e-02]
 [  1.30079379e-02   7.60796165e-02]
 [  1.37674731e-02   7.58066777e-02]
 [  1.45236772e-02   7.54154566e-02]
 [  1.52754280e-02   7.49179547e-02]
 

In [7]:
from scipy.integrate import ode

class MassSpringDamper(object):
    
    def __init__(self, m, k, b, F):
        self.m, self.k, self.b, self.F = m, k, b, F
        
    def f(self, t, xu):
        x, u = xu.tolist()
        dx = u
        du = (self.F - self.k*x - self.b*u) / self.m
        return [dx, du]
    
system = MassSpringDamper(m=m, k=k, b=b, F=F)
init_status = 0.0, 0.0
dt = 0.01

r = ode(system.f)
r.set_integrator('vode', method='bdf')
r.set_initial_value(init_status, 0)
t = []
result2 = [init_status]
while r.successful() and r.t + dt < 2:
    r.integrate(r.t + dt)
    t.append(r.t)
    result2.append(r.y)
    
result2 = np.array(result2)
np.allclose(result, result2)

True

In [8]:
class PID(object):
    
    def __init__(self, kp, ki, kd, dt):
        self.kp, self.ki, self.kd, self.dt = kp, ki, kd, dt
        self.last_error = None
        self.status = 0.0
        
    def update(self, error):
        p = self.kp * error
        i = self.ki * self.status
        if self.last_error is None:
            d = 0.0
        else:
            d = self.kd * (error - self.last_error) / self.dt
        self.status += error * self.dt
        self.last_error = error
        return p + i + d
    
def pid_control_system(kp, ki, kd, dt, target=1.0):
    system = MassSpringDamper(m=m, k=k, b=b, F=0.0)
    pid = PID(kp, ki, kd, dt)
    init_status = 0.0, 0.0
    
    r = ode(system.f)
    r.set_integrator('vode', method='bdf')
    r.set_initial_value(init_status, 0)
    
    t = [0]
    result = [init_status]
    F_arr = [0]
    while r.successful() and r.t + dt < 2.0:
        r.integrate(r.t + dt)
        err = target - r.y[0]
        F = pid.update(err)
        system.F = F
        t.append(r.t)
        result.append(r.y)
        F_arr.append(F)
        
    result = np.array(result)
    t = np.array(t)
    F_arr = np.array(F_arr)
    return t, F_arr, result

t, F_arr, result = pid_control_system(50.0, 100.0, 10.0, 0.001)
print u"控制力的终值:", F_arr[-1]

控制力的终值: 19.9434046824


In [10]:
%%time
from scipy import optimize

def eval_func(k):
    kp, ki, kd = k
    t, F_arr, result = pid_control_system(kp, ki, kd, 0.01)
    return np.sum(np.abs(result[:, 0] - 1.0))

kwargs = {
    "method":"L-BFGS-B",
    "bounds":[(10, 200), (10, 100), (1, 100)],
    "options":{"approx_grad": True
    }}

opt_k = optimize.basinhopping(eval_func, (10, 10, 10), niter=10, minimizer_kwargs=kwargs)
print opt_k.x

d:\python27\lib\site-packages\scipy\optimize\_basinhopping.py:279: OptimizeWarning: Unknown solver options: approx_grad
  return self.minimizer(self.func, x0, **self.kwargs)


[ 36.50234356  92.00961539   1.00003152]
Wall time: 1min 9s


In [12]:
kp, ki, kd = opt_k.x
t, F_arr, result = pid_control_system(kp, ki, kd, 0.01)
idx = np.argmin(np.abs(t - 0.5))
x, u = result[idx]
print "t={}, x={: g}, u={: g}".format(t[idx], x, u)

t=0.5, x= 0.990216, u= 1.13509
